<a href="https://colab.research.google.com/github/kplr-training/Data-Visualisation/blob/main/FR/Bokeh/ready_intro_to_plotting_with_bokeh.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction au traçage avec Bokeh

[Data set download](https://s3.amazonaws.com/bebi103.caltech.edu/data/gfmt_sleep.csv)

<hr />

In [ ]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"

In [ ]:
import numpy as np
import scipy.special
import pandas as pd

# Notre package principal de traçage (doit importer explicitement les sous-modules)
import bokeh.io
import bokeh.plotting

# Activer la visualisation des tracés Bokeh dans le notebook
bokeh.io.output_notebook()

## High-level and low-level plotting packages



In [ ]:
fname = os.path.join(data_path, "gfmt_sleep.csv")
df = pd.read_csv(fname, na_values="*")
df["insomnia"] = df["sci"] <= 16

df.groupby("insomnia")["percent correct"].median()

## Introduction au Bokeh


Il est important de noter que les sous-modules de Bokeh doivent souvent être importés explicitement, comme nous l'avons fait dans la cellule de code en haut de ce carnet. Notez aussi que si vous voulez que vos tracés soient visibles (et interactifs) dans le carnet, vous devez exécuter

```python
bokeh.io.output_notebook()
```

en haut du carnet (comme nous l'avons fait). Enfin, notez que nous devons également avoir installé l'extension PyViz JupyterLab, ce que vous devriez avoir fait lors de la configuration de votre ordinateur.

### La grammaire de Bokeh et notre premier graphe avec Bokeh

Construire un graphe avec Bokeh consiste en quatre étapes principales.

1. Création d'une figure sur laquelle seront placés les glyphes (symboles représentant des données, par exemple des points pour un diagramme de dispersion). Considérez cette figure comme une "toile" qui définit l'espace sur lequel vous allez "peindre" vos glyphes.
2. Définir une source de données (presque toujours un cadre de données ordonné) qui est la référence utilisée pour placer les glyphes.
3. Choisissez le type de glyphe que vous souhaitez.
4. Annoter les colonnes de la source de données pour déterminer comment elles sont utilisées pour placer (et éventuellement colorer, mettre à l'échelle, etc.) le glyphe.

Une fois ces étapes terminées, vous devez effectuer le rendu du graphique.

Passons en revue ces étapes pour générer un diagramme de dispersion de la confiance en cas d'erreur par rapport à la confiance en cas de correction pour l'étude sur l'appariement des visages en cas de privation de sommeil. 

1. La première étape consiste à créer une figure, notre "canevas". En créant la figure, nous réfléchissons implicitement au type de représentation que nous voulons pour nos données. En d'autres termes, nous devons spécifier les axes et leurs étiquettes. Nous pouvons également vouloir spécifier le titre de la figure, la présence ou non de lignes de grille et toutes sortes d'autres personnalisations. Naturellement, nous voulons aussi spécifier la forme de la figure.

(Presque) tout cela est accompli dans Bokeh en faisant un appel à `bokeh.plotting.figure()` avec les arguments appropriés.

In [ ]:
# Créer la figure, stockée dans la variable `p`
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label="confidence when correct",
    y_axis_label="condifence when incorrect",
)

There are many more keyword attributes you can assign, including [all of those listed in the Bokeh Plot class](https://bokeh.pydata.org/en/latest/docs/reference/models/plots.html#bokeh.models.plots.Plot) and [the additional ones listed in the Bokeh Figure class](https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure).

2. Now that we have set up our canvas, we can decide on the data source. We will use the data frame, `df` as our data source. 

3. We will choose dots (or circles) as our glyph. This kind of glyph requires that we specify  which column of the data source will serve to place the glyphs along the $x$-axis and which will serve to place the glyphs along the $y$-axis.

4. We choose the `'confidence when correct'` column to specify the $x$-coordinate of the glyph and the `'confidence when incorrect'` column to specify the $y$-coordinate. We already made this decision when we set up our axis labels, but we did not necessarily have to make that decision at that point.

Steps 2-4 are accomplished by calling one of the **glyph methods** of the Bokeh `Figure` instance, `p`. Since we are choosing dots, the appropriate method is `p.circle()`, and we use the `source`, `x`, and `y` kwargs to specify the positions of the glyphs.

In [ ]:
p.circle(
    source=df, 
    x="confidence when correct", 
    y="confidence when incorrect"
)

Maintenant que nous avons construit le graphe, nous pouvons le rendre dans le notebook en utilisant `bokeh.io.show()`.

In [ ]:
bokeh.io.show(p)

En regardant le graphique, vous remarquerez une barre d'outils à droite du graphique qui vous permet de zoomer et d'effectuer des panoramiques dans le graphique.

### Style de code dans les spécifications de tracé

Les spécifications de tracés impliquent souvent des appels à des fonctions avec de nombreux arguments de mots-clés pour spécifier le tracé, et cela peut devenir lourd sans un style clair. Vous pouvez développer votre propre style, peut-être en lisant à nouveau [l'article de blog de Trey Hunner] (http://treyhunner.com/2017/07/craft-your-python-like-poetry/). J'aime faire ce qui suit, ce qui n'est pas ce que [Black](https://github.com/psf/black) propose par défaut.

1. Mettre l'appel de la fonction, comme `p.circle(` ou `p = bokeh.plotting.figure(`sur la première ligne.
2. La parenthèse fermée de l'appel de fonction est sur sa propre ligne, sans indentation.
3. Tous les arguments sont donnés en tant que kwargs (même s'ils peuvent également être spécifiés en tant qu'arguments positionnels) à un niveau d'indentation.

Notez que vous ne pouvez pas utiliser le chaînage de méthodes lors de l'instanciation de figures ou du remplissage de glyphes.

Si vous adhérez à un style, votre code sera plus propre et plus facile à lire.

### Coloriage avec d'autres dimensions

Supposons que nous voulions faire le même graphique, mais avec des cercles orange pour les insomniaques et des cercles bleus pour les dormeurs normaux. Pour ce faire, nous tirons parti de deux fonctionnalités de Bokeh.

1. Nous pouvons faire plusieurs appels à `p.circle()` pour remplir de plus en plus de glyphes.
2. `p.circle()`, comme toutes les méthodes de glyphes, a de nombreux arguments de type mot-clé, y compris `color` et `legend_label`, qui nous permettront de colorer les glyphes et d'inclure une légende.

Nous pouvons parcourir en boucle la base de données groupée par "insomnie" et remplir les glyphes au fur et à mesure.

In [ ]:
# Pour plus de commodité
x = "confidence when correct"
y = "confidence when incorrect"

# Réaliser la figure
p = bokeh.plotting.figure(
    width=400, 
    height=300, 
    x_axis_label=x, 
    y_axis_label=y,
)

# Ajouter des glyphes
p.circle(
    source=df.loc[~df["insomnia"], :], 
    x=x, 
    y=y, 
    legend_label="normal sleepers",
)

p.circle(
    source=df.loc[df["insomnia"], :],
    x=x,
    y=y,
    color="orange",
    legend_label="insomniacs",
)

bokeh.io.show(p)

Nous avons obtenu le graphique que nous voulions, mais la légende est en contradiction avec les données. Heureusement, Bokeh nous permet de définir les attributs de la figure quand nous le souhaitons. Nous pouvons donc régler la position de la légende pour qu'elle soit dans le coin supérieur gauche. Nous allons également définir la `politique_de_clic` pour la légende à `'hide'`, ce qui cachera les glyphes si vous cliquez sur la légende, ce qui peut être pratique pour visualiser des tracés encombrés (bien que celui-ci ne soit pas encombré, en réalité). 

In [ ]:
p.legend.location = "top_left"
p.legend.click_policy = "hide"

bokeh.io.show(p)

## Ajouter des infobulles

L'interactivité de Bokeh est l'une de ses plus grandes forces. Pendant que nous traçons les confiances quand elles sont correctes et incorrectes, nous avons coloré le statut d'insomniaque. Nous pourrions également souhaiter avoir accès à d'autres informations dans notre source de données (bien rangée) si nous survolons un glyphe. Supposons que nous voulions connaître le numéro de participant, le sexe et l'âge de chaque participant. Nous pouvons demander à Bokeh de nous donner ces informations en ajoutant des **boutons d'aide** lorsque nous instancions la figure.

La syntaxe d'une info-bulle est une liste de 2-tuples, où chaque tuple représente l'info-bulle que vous voulez. La première entrée du tuple est l'étiquette et la seconde est la colonne de la source de données qui contient les valeurs. La deuxième entrée doit être précédée du symbole `@` signifiant qu'il s'agit d'un champ de la source de données et non d'un champ intrinsèque au graphe, qui est précédé du signe `$`. S'il y a des espaces dans l'en-tête de la colonne, il faut mettre le nom de la colonne entre accolades. (Voir la [documentation pour la spécification de l'infobulle](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#basic-tooltips) pour plus d'informations).

In [ ]:
# For convenience
x = "confidence when correct"
y = "confidence when incorrect"

# Réaliser la figure
p = bokeh.plotting.figure(
    width=400,
    height=300,
    x_axis_label=x,
    y_axis_label=y,
    tooltips=[
        ("p-number", "@{participant number}"),
        ("gender", "@gender"),
        ("age", "@age"),
    ],
)

# Ajouter des glyphes
p.circle(
    source=df.loc[~df["insomnia"], :], 
    x=x, 
    y=y, 
    legend_label="normal sleepers",
)

p.circle(
    source=df.loc[df["insomnia"], :],
    x=x,
    y=y,
    color="orange",
    legend_label="insomniacs",
)

p.legend.location = "top_left"
p.legend.click_policy = "hide"

bokeh.io.show(p)

## Sauvegarde des tracés Bokeh



In [ ]:
p.output_backend = "svg"

bokeh.io.show(p)

Cliquez ensuite sur l'icône du disque, puis revenez en arrière.

In [ ]:
p.output_backend = "canvas"

In [ ]:
bokeh.plotting.output_file("insomniac_confidence_correct.html")
bokeh.io.save(
    p, 
    title="Bokeh plot",
)

La page HTML qui en résulte possède toute l'interactivité de l'intrigue et vous pouvez, par exemple, l'envoyer par courrier électronique à vos collaborateurs pour qu'ils l'explorent.

## Computing environment

In [ ]:
%load_ext watermark
%watermark -v -p numpy,scipy,pandas,bokeh,jupyterlab